<a href="https://colab.research.google.com/github/leedain0301/Data_MiniProject_02/blob/main/MiniProject_02_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 컬럼 설명

#   train.csv / test.csv <br>
>  : 서울 지역의 740,000여개 거래 데이터, 아파트 거래일, 지역, 전용면적, 실 거래가 등의 정보

- **features**
  *   transaction_id : 아파트 거래 유일키키
  *   apartment_id : 아파트 아이디
  *   city : 도시
  *   dong : 동
  *   jibun : 지번
  *   apt : 아파트단지 이름
  *   addr_kr : 주소
  *   exclusive_use_area : 전용면적
  *   year_of_completion : 설립일자
  *   transaction_year_month : 거래년월
  *   transaction_date : 거래날짜
  *   floor : 층
  *   transaction_real_price : 실거래가(train 만 존재)



<br>

---

<br>









In [ ]:
!pip install category_encoders


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import sklearn
from sklearn.linear_model    import ElasticNet, Lasso
from sklearn.ensemble        import GradientBoostingRegressor, RandomForestRegressor
from sklearn.kernel_ridge    import KernelRidge
from sklearn.pipeline        import make_pipeline
from sklearn.preprocessing   import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics         import mean_squared_error, accuracy_score
from sklearn.preprocessing   import StandardScaler
from sklearn.impute          import SimpleImputer
from category_encoders       import OneHotEncoder, OrdinalEncoder
from sklearn.ensemble        import RandomForestClassifier

from xgboost.sklearn import XGBClassifier, XGBRegressor
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
import sys

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/02.Colab Notebooks/MiniProject_02/tarin_clean.csv')
test = pd.read_csv('/content/drive/MyDrive/02.Colab Notebooks/MiniProject_02/test_clean.csv')
submission_data = pd.read_csv('/content/drive/MyDrive/02.Colab Notebooks/MiniProject_02/submission.csv')

print('train.csv. Shape: ',train.shape)
print('test.csv. Shape: ', test.shape)

train.csv. Shape:  (593828, 9)
test.csv. Shape:  (3911, 8)


In [ ]:
train = train.drop('Unnamed: 0', axis=1)
test = test.drop('Unnamed: 0', axis=1)
# train['transaction_real_price'] = np.expm1(train['transaction_real_price'])
train.head()

,dong,exclusive_use_area,year_of_completion,transaction_date,floor,transaction_real_price,year,month
0,상계동,59.95,1998,11~20,2.944439,10.221978,2008,3
1,쌍문동,84.53,1987,21~30,2.772589,10.288784,2009,9
2,창동,58.01,1990,1~10,2.833213,10.165890,2008,5
3,둔촌동,58.33,2004,11~20,2.639057,10.034560,2009,1
4,구로동,49.94,1987,21~31,2.197225,10.146473,2015,8


In [ ]:
baseline_score = train['transaction_real_price'].value_counts(normalize=True).max()
print("baseline score:", baseline_score)

baseline score: 0.011016658022188243


### **< 데이터셋 >**

---


In [ ]:
train, val = train_test_split(
    train, train_size=0.80,  random_state=42
)

train.shape, val.shape, test.shape

((475062, 8), (118766, 8), (3911, 7))

In [ ]:
train['exclusive_use_area']=train['exclusive_use_area'].astype(int)
train['floor']=train['floor'].astype(int)

test['exclusive_use_area']=test['exclusive_use_area'].astype(int)
test['floor']=test['floor'].astype(int)

In [ ]:
target = 'transaction_real_price'
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

### **< Data processing >**

---


In [ ]:
encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train) 
X_val_encoded = encoder.transform(X_val) 

boosting = XGBRegressor(
    n_estimators=100,
    learning_rate=0.2,
    subsample=0.8,
    colsample_bytree=0.8,
)

eval_set = [(X_train_encoded, y_train), (X_val_encoded, y_val)]

boosting.fit(X_train_encoded, y_train, eval_set=eval_set, early_stopping_rounds=50)

[06:19:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:8.1273	validation_1-rmse:8.12289
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:6.50572	validation_1-rmse:6.49904
[2]	validation_0-rmse:5.21102	validation_1-rmse:5.20422
[3]	validation_0-rmse:4.17572	validation_1-rmse:4.16355
[4]	validation_0-rmse:3.34708	validation_1-rmse:3.3343
[5]	validation_0-rmse:2.68545	validation_1-rmse:2.67074
[6]	validation_0-rmse:2.15793	validation_1-rmse:2.14278
[7]	validation_0-rmse:1.7382	validation_1-rmse:1.72308
[8]	validation_0-rmse:1.4057	validation_1-rmse:1.39068
[9]	validation_0-rmse:1.14181	validation_1-rmse:1.12706
[10]	validation_0-rmse:0.934197	validation_1-rmse:0.919737
[11]	validation_0-rmse:0.772593	validation_1-rmse:0.758573
[12]	validation_0-rmse:0.647023	validati

XGBRegressor(colsample_bytree=0.8, learning_rate=0.2, subsample=0.8)

In [ ]:
print("검증 accuracy", boosting.score(X_val_encoded, y_val))

검증 accuracy 0.8111997094582533


In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train_encoded, y_train)

ValueError: ignored